# MLFlow with Linear Regression example

Based on https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

In [6]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [7]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [4]:
# Read the wine-quality csv file from the URL
csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]



In [15]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
129,6.1,0.38,0.15,1.8,0.072,6.0,19.0,0.99550,3.42,0.57,9.4
60,8.8,0.40,0.40,2.2,0.079,19.0,52.0,0.99800,3.44,0.64,9.2
338,12.4,0.49,0.58,3.0,0.103,28.0,99.0,1.00080,3.16,1.00,11.5
1513,6.4,0.56,0.15,1.8,0.078,17.0,65.0,0.99294,3.33,0.60,10.5
28,7.1,0.71,0.00,1.9,0.080,14.0,35.0,0.99720,3.47,0.55,9.4


In [16]:
train_y.head()

,quality
129,5
60,5
338,6
1513,6
28,5


In [20]:
alpha = 0.65
l1_ratio = 0.65

In [23]:
mlflow.set_experiment(experiment_id='1') # this will assign the run (below) to the expiment defined in MLFLow UI

<Experiment: artifact_location='file:///home/wojtek/Workspace/max-ml/datascience-trainings/mlflow-training/mlruns/1', creation_time=1667846259947, experiment_id='1', last_update_time=1667846832864, lifecycle_stage='active', name='Linear Regression Training', tags={}>

In [22]:

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":
        print('store type == file')
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        print('store type != file')
        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.650000, l1_ratio=0.650000):
  RMSE: 0.7353428747057151
  MAE: 0.6199788575627271
  R2: 0.06464427714847776
store type != file
